# Лабораторная работа №6

**1**.	Таблица moscow-buildings.csv содержит информацию о годах постройки жилых домов в Москве.

1)	Загрузите базу данных moscow-buildings.csv и сохраните ее как df_build. Удалите из базы данных строки, в которых значения года постройки не указаны («н.д.»), а после превратите столбец house_year в числовой.

2)	Посмотрите на минимальное и максимальное значения года постройки. Разумны ли они? Если нет, уберите из базы данных «странные» значения года постройки. Подсчитайте количество удаленных записей.

3)	Выберите из базы данных строки, соответствующие Басманному району и сохраните их в базу basm_data.

4)	Сгруппируйте данные в базе по районам и посмотрите, сколько домов в базе относятся к каждому району.

5)	Найдите средний возраст дома (возраст – в годах, считаем от текущего года) по каждому району. Подсказка: сначала добавьте переменную house_age («возраст дома»).

6)	Найдите самый старый дом и выведите информацию о нем.

In [11]:
# 1)
import pandas as pd
from datetime import datetime

currentYear = datetime.now().year

df_build = pd.read_csv('moscow-buildings.csv', index_col=0)
#df_build.info()
rows_before = len(df_build.axes[0])
print(f'Число строк до удаления тех, что содержать "н.д.": {rows_before}')

# Удаляем строки, содержащие "н.д."
df_build = df_build[(df_build.house_year != 'н.д.')]
#df_build.info()
rows_after = len(df_build.axes[0])
print(f'Число строк после удаления тех, что содержать "н.д.": {rows_after}\n'
      f'Число удаленных строк: {rows_before - rows_after}')

# Меняем тип данных столбца house_year
datatypes_before = df_build.dtypes['house_year']
print(f'\nТип данных в столбце house_year до изменений: {datatypes_before}')
df_build = df_build.astype({'house_year': int})
datatypes_after = df_build.dtypes['house_year']
print(f'Тип данных в столбце house_year после изменения в числовой: {datatypes_after}')


Число строк до удаления тех, что содержать "н.д.": 29464
Число строк после удаления тех, что содержать "н.д.": 26312
Число удаленных строк: 3152

Тип данных в столбце house_year до изменений: object
Тип данных в столбце house_year после изменения в числовой: int32


In [12]:
# 2)
# Получаем максимальный и минимальный год постройки зданий
maxValues = df_build['house_year'].max()
minValues = df_build['house_year'].min()
print(f'\nМаксимальный год постройки здания - {maxValues}\n'
      f'Минимальный год постройки здания - {minValues}')
# Год основания Москвы - 1147 [https://www.mos.ru/news/item/22507073/]
df_build_copy = df_build.copy()
# Удаляем строки, содержащие годы постройки до основания Москвы и больше нынешнего года
df_build = df_build.drop(df_build[(df_build['house_year'] < 1147) | (df_build['house_year'] > currentYear)].index)
rows_after_years = len(df_build)
print(f'\nЧисло строк после удаления тех, что содержали некорректные данные в столбце house_year: {rows_after_years}\n'
      f'Число удаленных строк: {rows_after - rows_after_years}')


Максимальный год постройки здания - 19781962
Минимальный год постройки здания - 5

Число строк после удаления тех, что содержали некорректные данные в столбце house_year: 21479
Число удаленных строк: 4833


In [13]:
# 3)
# Выбираем из базы данных строки, соответствующие Басманному району
#print(df_build['area_name'])
basm_data = df_build_copy.copy()
basm_data = basm_data.drop(basm_data[(basm_data['area_name'] != 'муниципальный округ Басманный')].index)
print(f'\nЧисло строк, содержащих информацию о Басманном районе: {len(basm_data)}')


Число строк, содержащих информацию о Басманном районе: 636


In [14]:
# 4)
# Группировка домов по району
grouped = df_build.groupby('area_name')['house_id'].size()
print(f'\nЧисло домов по районам: \n{grouped}')


Число домов по районам: 
area_name
муниципальный округ Алексеевский    212
муниципальный округ Алтуфьевский    114
муниципальный округ Арбат           134
муниципальный округ Аэропорт        186
муниципальный округ Бабушкинский    251
                                   ... 
муниципальный округ Южное Тушино    320
муниципальный округ Южнопортовый    200
муниципальный округ Якиманка        105
муниципальный округ Ярославский     177
муниципальный округ Ясенево         152
Name: house_id, Length: 108, dtype: int64


In [15]:
# 5)
# Находим средний возраст дома по районам
df_build = df_build.assign(house_age=currentYear - df_build['house_year'])
grouped_age = df_build.groupby('area_name')['house_age'].mean()
print(grouped_age)

area_name
муниципальный округ Алексеевский    60.155660
муниципальный округ Алтуфьевский    51.280702
муниципальный округ Арбат           92.462687
муниципальный округ Аэропорт        60.102151
муниципальный округ Бабушкинский    55.123506
                                      ...    
муниципальный округ Южное Тушино    53.471875
муниципальный округ Южнопортовый    60.095000
муниципальный округ Якиманка        74.533333
муниципальный округ Ярославский     51.028249
муниципальный округ Ясенево         42.664474
Name: house_age, Length: 108, dtype: float64


In [16]:
# 6)
# Находим самый старый дом и выводим по нему инфу
oldest = df_build['house_year'].min()
oldest_house = df_build.loc[df_build['house_year'] == oldest]
print('\nИнформация о самом старом доме:\n'
      f'{oldest_house}')


Информация о самом старом доме:
               full_address        city address street_prefix street_name  \
area_id                                                                     
2281124  г Зеленоград д.811  Зеленоград   д.811           NaN         NaN   

        building  house_id                           area_name  house_year  \
area_id                                                                      
2281124      NaN   7742576  муниципальный округ Старое Крюково        1273   

         house_age  
area_id             
2281124        750  


**2.**	Загрузите таблицу titanic.csv и найдите ответы на следующие вопросы. Если ответ нужно приводить в процентах, то это должно быть число в интервале от 0 до 100, округленное до двух знаков, знак процента не нужен.  

1)	Посчитайте долю выживших пассажиров.ok

2)	Какое количество мужчин и женщин ехало на корабле? Определите долю выживших женщин и выживших мужчин.ok

3)	Какую долю пассажиры первого, второго и третьего классов составляли среди всех пассажиров? Определите долю выживших пассажиров каждого класса.ok

4)	 Посчитайте среднее и медиану возраста пассажиров. Посчитайте среднее и медиану возраста выживших и невыживших пассажиров.

5)	Добавьте столбцы FirstNameList с именем пассажира. Обратите внимание, что встречается разный формат хранения имен, в том числе существуют отличия для мужчин и женщин. Имя может быть составное (из нескольких имен) - рассмотрите оба варианта - составное имя целиком и составное имя как отдельные имена. Какое самое популярное женское имя на корабле? Какое самое популярное мужское имя на корабле?

6)	Приведите список уникальных (неповторяющихся, только один человек имеет это имя) имен пассажиров, отдельно мужских и женских. Рассмотрите оба варианта - составное имя целиком и составное имя как отдельные имена. Выведите таблицу с этими пассажирами.

7)	Приведите множество всех имен пассажиров, отдельно мужских и женских. Рассмотрите оба варианта - составное имя целиком и составное имя как отдельные имена.

8)	Приведите список имен, которые одновременно встречаются у мужчин, и у женщин. Решите задачу, если рассматривать составные имена как одно имя, и если рассматривать составные имена отдельно. Выведите таблицу пассажирами, имеющими эти имена.

In [56]:
titanic_df = pd.read_csv('titanic.csv', index_col=0)
titanic_df
#titanic_df.info()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S


In [26]:
#1

surviving_passengers = titanic_df["Survived"].sum() / len(titanic_df)
print(f"Доля выживших пассажиров: {surviving_passengers * 100:.2f}%")

Доля выживших пассажиров: 38.38%


In [55]:
gender_counts = titanic_df["Sex"].value_counts()
survived_pass = titanic_df[titanic_df["Survived"] == 1]["Sex"].value_counts()
survived_percentage = (survived_pass / gender_counts) * 100

print(f"Количество мужчин на корабле: {gender_counts['male']}")
print(f"Количество женщин на корабле:{gender_counts['female']}")

print(f"\nКоличество выживших мужчин: {survived_pass['male']}")
print(f"Количество выживших женщин: {survived_pass['female']}")

print(f"\nДоля выживших мужчин: {survived_percentage['male']:.2f}")
print(f"Доля выживших женщин: {survived_percentage['female']:.2f}")

Количество мужчин на корабле: 577
Количество женщин на корабле:314

Количество выживших мужчин: 109
Количество выживших женщин: 233

Доля выживших мужчин: 18.89%
Доля выживших женщин: 74.20%


In [68]:
passenger_classes = titanic_df["Pclass"]
count_classes = titanic_df["Pclass"].value_counts()
survived_pass_by_classes = titanic_df[titanic_df["Survived"] == 1]["Pclass"].value_counts()

print(f"Доля пассажиров каждого класса среди всех пассажиров: "
      f"\nПервый класс: {count_classes[1] / len(titanic_df) * 100:.2f}"
      f"\nВторой класс: {count_classes[2] / len(titanic_df) * 100:.2f}"
      f"\nТретий класс: {count_classes[3] / len(titanic_df) * 100:.2f}")
print(f"Доля выживших пассажиров каждого класса среди всех пассажиров: "
      f"\nПервый класс: {survived_pass_by_classes[1] / sum(survived_pass_by_classes) * 100:.2f}"
      f"\nВторой класс: {survived_pass_by_classes[2] / sum(survived_pass_by_classes) * 100:.2f}"
      f"\nТретий класс: {survived_pass_by_classes[3] / sum(survived_pass_by_classes) * 100:.2f}")

Доля пассажиров каждого класса среди всех пассажиров: 
Первый класс: 24.24
Второй класс: 20.65
Третий класс: 55.11
Доля выживших пассажиров каждого класса среди всех пассажиров: 
Первый класс: 39.77
Второй класс: 25.44
Третий класс: 34.80


4)	 Посчитайте среднее и медиану возраста пассажиров. Посчитайте среднее и медиану возраста выживших и невыживших пассажиров.

5)	Добавьте столбцы FirstNameList с именем пассажира. Обратите внимание, что встречается разный формат хранения имен, в том числе существуют отличия для мужчин и женщин. Имя может быть составное (из нескольких имен) - рассмотрите оба варианта - составное имя целиком и составное имя как отдельные имена. Какое самое популярное женское имя на корабле? Какое самое популярное мужское имя на корабле?

6)	Приведите список уникальных (неповторяющихся, только один человек имеет это имя) имен пассажиров, отдельно мужских и женских. Рассмотрите оба варианта - составное имя целиком и составное имя как отдельные имена. Выведите таблицу с этими пассажирами.

7)	Приведите множество всех имен пассажиров, отдельно мужских и женских. Рассмотрите оба варианта - составное имя целиком и составное имя как отдельные имена.

8)	Приведите список имен, которые одновременно встречаются у мужчин, и у женщин. Решите задачу, если рассматривать составные имена как одно имя, и если рассматривать составные имена отдельно. Выведите таблицу пассажирами, имеющими эти имена.

In [74]:
age_mean = titanic_df["Age"].dropna().mean()
age_median = titanic_df["Age"].dropna().median()

print(f"Средний возраст всех пассажиров:{age_mean:.2f}")
print(f"Медиана возраста всех пассажиров: {age_median}")

age_mean_survived = titanic_df[titanic_df["Survived"] == 1]["Age"].dropna().mean()
age_median_survived = titanic_df[titanic_df["Survived"] == 1]["Age"].dropna().median()

print(f"Средний возраст всех выживших пассажиров:{age_mean_survived:.2f}")
print(f"Медиана возраста всех выживших пассажиров: {age_median_survived}")

age_mean_not_survived = titanic_df[titanic_df["Survived"] == 0]["Age"].dropna().mean()
age_median_not_survived = titanic_df[titanic_df["Survived"] == 0]["Age"].dropna().median()

print(f"Средний возраст всех не выживших пассажиров:{age_mean_not_survived:.2f}")
print(f"Медиана возраста всех не выживших пассажиров: {age_median_not_survived}")

Средний возраст всех пассажиров:29.70
Медиана возраста всех пассажиров: 28.0
Средний возраст всех выживших пассажиров:28.34
Медиана возраста всех выживших пассажиров: 28.0
Средний возраст всех не выживших пассажиров:30.63
Медиана возраста всех не выживших пассажиров: 28.0


In [125]:
# 5)	Добавьте столбцы FirstNameList с именем пассажира. Обратите внимание, что встречается разный формат хранения имен, в том числе существуют отличия для мужчин и женщин. Имя может быть составное (из нескольких имен) - рассмотрите оба варианта - составное имя целиком и составное имя как отдельные имена. Какое самое популярное женское имя на корабле? Какое самое популярное мужское имя на корабле?

import pandas as pd

# Загрузите данные из файла titanic.csv или используйте свои данные
titanic_df = pd.read_csv('titanic.csv')

# Создайте столбец "FirstNameList" с именем пассажира
titanic_df['FirstNameList'] = titanic_df['Name'].str.split(',').str.get(1).str.strip()
titanic_df['FirstNameList'] = titanic_df['FirstNameList'].str.split('.').str.get(1).str.strip()

# Отобразим первые несколько строк для проверки
print(titanic_df['FirstNameList'].head())

# Популярные имена у женщин
female_names = titanic_df[titanic_df['Sex'] == 'female']['FirstNameList']
most_common_female_name = female_names.mode().iloc[0]

# Популярные имена у мужчин
male_names = titanic_df[titanic_df['Sex'] == 'male']['FirstNameList']
most_common_male_name = male_names.mode().iloc[0]

print(f"Самое популярное женское имя на корабле: {most_common_female_name}")
print(f"Самое популярное мужское имя на корабле: {most_common_male_name}")

titanic_df['FirstNameList']

0                              Owen Harris
1    John Bradley (Florence Briggs Thayer)
2                                    Laina
3            Jacques Heath (Lily May Peel)
4                            William Henry
Name: FirstNameList, dtype: object
Самое популярное женское имя на корабле: Mary
Самое популярное мужское имя на корабле: John


0                                Owen Harris
1      John Bradley (Florence Briggs Thayer)
2                                      Laina
3              Jacques Heath (Lily May Peel)
4                              William Henry
                       ...                  
886                                   Juozas
887                           Margaret Edith
888                 Catherine Helen "Carrie"
889                              Karl Howell
890                                  Patrick
Name: FirstNameList, Length: 891, dtype: object

In [153]:
# 6)	Приведите список уникальных (неповторяющихся, только один человек имеет это имя) имен пассажиров, отдельно мужских и женских. Рассмотрите оба варианта - составное имя целиком и составное имя как отдельные имена. Выведите таблицу с этими пассажирами.

import pandas as pd

# Загрузите данные из файла titanic.csv или используйте свои данные
titanic_df = pd.read_csv('titanic.csv')

# Создайте столбец "FirstNameList" с именем пассажира
titanic_df['FirstNameList'] = titanic_df['Name'].str.split(',').str.get(1).str.strip()
titanic_df['FirstNameList'] = titanic_df['FirstNameList'].str.split('.').str.get(1).str.strip()

# Разделите имена на отдельные части
titanic_df['FirstNameList'] = titanic_df['FirstNameList'].str.split()

# Уникальные имена у женщин (составные имена)
unique_female_full_names = [name for name in titanic_df[titanic_df['Sex'] == 'female']['FirstNameList'] if list(titanic_df[titanic_df['Sex'] == 'female']['FirstNameList']).count(name) == 1]

# Уникальные имена у женщин (отдельные имена)
unique_female_names = [name for names in titanic_df[titanic_df['Sex'] == 'female']['FirstNameList'] for name in names if list(titanic_df[titanic_df['Sex'] == 'female']['FirstNameList']).count(name) == 1]

# Уникальные имена у мужчин (составные имена)
unique_male_full_names = [name for name in titanic_df[titanic_df['Sex'] == 'male']['FirstNameList'] if list(titanic_df[titanic_df['Sex'] == 'male']['FirstNameList']).count(name) == 1]

# Уникальные имена у мужчин (отдельные имена)
unique_male_names = [name for names in titanic_df[titanic_df['Sex'] == 'male']['FirstNameList'] for name in names if list(titanic_df[titanic_df['Sex'] == 'male']['FirstNameList']).count(name) == 1]

# Выведите таблицы с этими пассажирами
unique_female_full_names_df = titanic_df[titanic_df['FirstNameList'].apply(lambda x: any(name in unique_female_full_names for name in x))]
unique_female_names_df = titanic_df[titanic_df['FirstNameList'].apply(lambda x: any(name in unique_female_names for name in x))]
unique_male_full_names_df = titanic_df[titanic_df['FirstNameList'].apply(lambda x: any(name in unique_male_full_names for name in x))]
unique_male_names_df = titanic_df[titanic_df['FirstNameList'].apply(lambda x: any(name in unique_male_names for name in x))]

print("Уникальные имена у женщин (составные имена):")
print(unique_female_full_names)

print("\nУникальные имена у женщин (отдельные имена):")
print(unique_female_names)

print("\nУникальные имена у мужчин (составные имена):")
print(unique_male_full_names)

print("\nУникальные имена у мужчин (отдельные имена):")
print(unique_male_names)




Уникальные имена у женщин (составные имена):
[['John', 'Bradley', '(Florence', 'Briggs', 'Thayer)'], ['Laina'], ['Jacques', 'Heath', '(Lily', 'May', 'Peel)'], ['Oscar', 'W', '(Elisabeth', 'Vilhelmina', 'Berg)'], ['Nicholas', '(Adele', 'Achem)'], ['Marguerite', 'Rut'], ['Hulda', 'Amanda', 'Adolfina'], ['(Mary', 'D', 'Kingcome)'], ['Julius', '(Emelia', 'Maria', 'Vandemoortele)'], ['Fatima'], ['Anna', '"Annie"'], ['Torborg', 'Danira'], ['Carl', 'Oscar', '(Selma', 'Augusta', 'Emilia', 'Johansson)'], ['William', 'Augustus', '(Marie', 'Eugenie)'], ['Mary', 'Agatha'], ['Augusta', 'Maria'], ['Jamila'], ['Johan', '(Johanna', 'Persdotter', 'Larsson)'], ['William', 'John', 'Robert', '(Dorothy', 'Ann', 'Wonnacott)'], ['Simonne', 'Marie', 'Anne', 'Andree'], ['Margaret', 'Delia'], ['Bridget'], ['Josef', '(Josefine', 'Franchi)'], ['Henry', 'Sleeper', '(Myna', 'Haxtun)'], ['Lizzie', '(Elizabeth', 'Anne', 'Wilkinson)'], ['Emily'], ['Constance', 'Mirium'], ['Amelie'], ['(Elizabeth', 'Ramell)'], ['Erna',

In [149]:
# 7)Приведите множество всех имен пассажиров, отдельно мужских и женских. Рассмотрите оба варианта - составное имя целиком и составное имя как отдельные имена.

import pandas as pd

# Загрузите данные из файла titanic.csv или используйте свои данные
titanic_df = pd.read_csv('titanic.csv')

# Создайте столбец "FirstNameList" с именем пассажира
titanic_df['FirstNameList'] = titanic_df['Name'].str.split(',').str.get(1).str.strip()
titanic_df['FirstNameList'] = titanic_df['FirstNameList'].str.split('.').str.get(1).str.strip()

# Разделите имена на отдельные части
titanic_df['FirstNameList'] = titanic_df['FirstNameList'].str.split()

# Множество всех имен у женщин (составные имена)
set_female_full_names = set(name for name_list in titanic_df[titanic_df['Sex'] == 'female']['FirstNameList'] for name in name_list)

# Множество всех имен у женщин (отдельные имена)
set_female_names = set(name for names in titanic_df[titanic_df['Sex'] == 'female']['FirstNameList'] for name in names)

# Множество всех имен у мужчин (составные имена)
set_male_full_names = set(name for name_list in titanic_df[titanic_df['Sex'] == 'male']['FirstNameList'] for name in name_list)

# Множество всех имен у мужчин (отдельные имена)
set_male_names = set(name for names in titanic_df[titanic_df['Sex'] == 'male']['FirstNameList'] for name in names)

print("Множество всех имен у женщин (составные имена):")
print(set_female_full_names)

print("\nМножество всех имен у женщин (отдельные имена):")
print(set_female_names)

print("\nМножество всех имен у мужчин (составные имена):")
print(set_male_full_names)

print("\nМножество всех имен у мужчин (отдельные имена):")
print(set_male_names)






Множество всех имен у женщин (составные имена):
{'Simonne', 'Persdotter', 'Ann', 'Parrish', 'Harriet', 'Thayer)', 'Gretchen', 'Hildur', 'Courtenay', 'Gerda', 'Wilhelm', 'Viktoria', 'Thamine', 'Emil', 'Hanora', 'Konstantia', 'Satode', 'Adolfina', 'Jessie', 'Heath', 'Hitchcock)', 'Ada', 'James', 'Frederic', 'Achem)', 'Joel', '(Sylvia', 'Adele', 'Sam', 'Francoise', 'Celia', 'Harbaugh)', 'Anna', 'Iris', 'Telma', 'E', 'Bradley', '(Charlotte', 'R', 'Jefferys)', 'Clarke)', '(Dorothy', '(Lily', 'Phyllis', 'Lizzie', 'Norman', 'Milley)', 'Johan', 'Luise', 'Monypeny', 'Razi)', 'Ruth', 'DeLaudeniere', 'of', 'Natalia', 'K', 'Hedwig', '("Mary")', 'Edith', 'Martha', '(Bessie', 'Adelia', 'Watson)', 'Margit', 'Godfrey)', '(Mantoura', 'Frank', 'Slow)', 'Wright)', 'Frances', 'Ileen', 'Alexander)', 'Lafargue)', 'Viola', '(Jessie', 'Josef', '(Bertha', 'Jean', 'Amy', 'McMillan)', 'Brown)', 'Natalie', 'Walton', 'Soto', 'Eva', 'Bertha', '(Ada', '(Alfrida', '(Alma', 'Birkhardt', '(Mary)', 'Phillips', 'Gregg)',

In [91]:
# 8) Приведите список имен, которые одновременно встречаются у мужчин, и у женщин. Решите задачу, если рассматривать составные имена как одно имя, и если рассматривать составные имена отдельно. Выведите таблицу пассажирами, имеющими эти имена.
import pandas as pd

# Загрузите данные из файла titanic.csv или используйте свои данные
titanic_df = pd.read_csv('titanic.csv')

# Создайте столбец "FirstNameList" с именем пассажира
titanic_df['FirstNameList'] = titanic_df['Name'].str.split(',').str.get(1).str.strip()
titanic_df['FirstNameList'] = titanic_df['FirstNameList'].str.split('.').str.get(1).str.strip()

# Разделите имена на отдельные части
titanic_df['FirstNameList'] = titanic_df['FirstNameList'].str.split()

# Множество всех имен у женщин
set_female_full_names = set(name for name_list in titanic_df[titanic_df['Sex'] == 'female']['FirstNameList'] for name in name_list)

# Множество всех имен у мужчин
set_male_full_names = set(name for name_list in titanic_df[titanic_df['Sex'] == 'male']['FirstNameList'] for name in name_list)

# Имена, которые встречаются как у мужчин, так и у женщин
common_names = set_female_full_names.intersection(set_male_full_names)

# Выведите таблицу пассажиров, имеющих эти имена
passengers_with_common_names = titanic_df[titanic_df['FirstNameList'].apply(lambda x: any(name in common_names for name in x))]


print("Имена, которые встречаются у мужчин и женщин:")
print(common_names)

print("Пассажиры с этими именами:")
print(passengers_with_common_names[['Name', 'Sex']])

Имена, которые встречаются как у мужчин, так и у женщин (составное имя целиком):
set()

Имена, которые встречаются как у мужчин, так и у женщин (отдельные имена):
set()

Пассажиры с общими именами (составное имя целиком):
Empty DataFrame
Columns: [PassengerId, Survived, Pclass, Name, Sex, Age, SibSp, Parch, Ticket, Fare, Cabin, Embarked, FullNameList, FirstName]
Index: []

Пассажиры с общими именами (отдельные имена):
Empty DataFrame
Columns: [PassengerId, Survived, Pclass, Name, Sex, Age, SibSp, Parch, Ticket, Fare, Cabin, Embarked, FullNameList, FirstName]
Index: []
